## E03. Train model from all the available data
#### 0. Load required libraries, site dependant constants and utility functions.

In [2]:
from os import listdir
from os.path import isfile, join
from lightgbm.sklearn import LGBMClassifier
import numpy as np
import seaborn as sns
import datetime
from sklearn import metrics
import joblib
# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Load sampled eopatches:

In [3]:
# pick dataset_name
input_dataset_file_name = '_sptl_smpl_20000_pr_patch_blanced_only_wat_1_min_wat_50/'

dataset_filepath_2013 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2013_05-2013_10{}'.format(input_dataset_file_name)
dataset_filepath_2014 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2014_05-2014_10{}'.format(input_dataset_file_name)
dataset_filepath_2015 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2015_05-2015_10{}'.format(input_dataset_file_name)
dataset_filepath_2016 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2016_05-2016_10{}'.format(input_dataset_file_name)
dataset_filepath_2017 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2017_05-2017_10{}'.format(input_dataset_file_name)
dataset_filepath_2018 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2018_05-2018_10{}'.format(input_dataset_file_name)
dataset_filepath_2019 = '../../data/EOPatches/LANDSAT_8/UPE_PROMICE/UTM_22N/2019_05-2019_10{}'.format(input_dataset_file_name)

dataset_path_list = [dataset_filepath_2013,
                dataset_filepath_2014,
                dataset_filepath_2015,
                dataset_filepath_2016,
                dataset_filepath_2017,
                dataset_filepath_2018,
                dataset_filepath_2019]

In [4]:
%%time
train_eopatches= []
train_eopatches_filenames = []

for eopatches_filepath in dataset_path_list:

    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    list_in_chunks = io_utils.chunkIt(list_of_available_patches, 1 ) # number of chunks- 1 bc all can go
    for chunk in list_in_chunks:
        #print('Doing now following eopatches:', chunk )
        for eopatch_name in chunk:
            train_eopatches_filenames.append(eopatches_filepath+eopatch_name)
            train_eopatches.append(EOPatch.load(eopatches_filepath+eopatch_name, lazy_loading=True))
print(len(train_eopatches))

269
Wall time: 2.44 s


####  2. Fetch and organise dataset

In [6]:
# Definition of the train and test patch IDs

# Set the features and the labels for train and test sets
features_train = np.array([eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'] for eopatch in train_eopatches if eopatch.data['DATASET_CLD_200_dil_6_str2_SAMPLED'].size > 0])
#features_train (number_of_patches,(time,width,height, bands))
labels_train = np.array([eopatch.mask['WATER_MASK_ST_025_SAMPLED'] for eopatch in train_eopatches if eopatch.mask['WATER_MASK_ST_025_SAMPLED'].size > 0])

In [8]:
#reshape to  (number_of_patches x time, width, height, bands)
features_train_stacked = np.vstack(features_train)
labels_train_stacked = np.vstack(labels_train)

# get shape
p_train_x_time, w, h, b = features_train_stacked.shape

features_train = features_train_stacked.reshape(p_train_x_time * w * h, b)
labels_train = labels_train_stacked.reshape(p_train_x_time * w * h, 1)

####  3. Data statistics:

In [11]:
#to compare old data distribution (little less )

print('TRAIN DATA:')
tot_obs = np.size(labels_train)
print('Total observations: ', tot_obs )

tot_wat = np.count_nonzero(labels_train)
print('Total water count', tot_wat )

ratio_wat_vs_non_wat = tot_wat/ tot_obs
print('Percent of water feature among data:', ratio_wat_vs_non_wat )

TRAIN DATA:
Total observations:  37320000
Total water count 7454798
Percent of water feature among data: 0.19975342979635585


####  4. Model training

In [12]:
%%time

# Set up training classes
labels_unique = np.unique(labels_train)

# Set up the model
model = LGBMClassifier(boosting='gbdt',
    objective='binary',
    learning_rate=0.1,
    metric='binary_logloss', #mae: mean absolute error mse: mean squared error
    #class_weight= 'balanced' 
                       
)

# train the model
model.fit(features_train, labels_train)


C:\Users\Kamil\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Kamil\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 2min 18s


LGBMClassifier(boosting='gbdt', boosting_type='gbdt', class_weight=None,
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, metric='binary_logloss', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=31, objective='binary', random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

####  5. Save model

In [13]:
# uncomment to save the model
model_base_name = 'LGBMClassifier_model_20000_unbalanced_WATER_only_MODIS_test_0_fixed_lr_01_min_wat_50'
joblib.dump(model, './{}.pkl'.format(model_base_name))

['./LGBMClassifier_model_20000_unbalanced_WATER_only_MODIS_test_0_fixed_lr_01_min_wat_50.pkl']

####  6. Validate model with training data

In [15]:
# predict the train labels
plabels_train = model.predict(features_train)
print('TRAIN:')
print('Classification accuracy {:.1f}%'.format(100 * metrics.accuracy_score(labels_train, plabels_train)))
print('Classification F1-score {:.1f}%'.format(100 * metrics.f1_score(labels_train, plabels_train, average='weighted')))

TRAIN:
Classification accuracy 84.3%
Classification F1-score 82.2%


In [17]:
class_labels = np.unique(labels_train)
print(class_labels)
class_names = ['non-water', 'water']
class_names

[False  True]


['non-water', 'water']

In [18]:

f1_scores = metrics.f1_score(labels_train, plabels_train, labels=class_labels, average=None)
recall = metrics.recall_score(labels_train, plabels_train, labels=class_labels, average=None)
precision = metrics.precision_score(labels_train, plabels_train, labels=class_labels, average=None) 

print('             Class              =  F1  | Recall | Precision')
print('         --------------------------------------------------')
for idx, lulctype in enumerate([class_names[idx] for idx in class_labels]):
    print('         * {0:20s} = {1:2.1f}% |  {2:2.1f}%  | {3:2.1f}%'.format(lulctype, 
                                                                         f1_scores[idx] * 100, 
                                                                         recall[idx] * 100, 
                                                                         precision[idx] * 100))

             Class              =  F1  | Recall | Precision
         --------------------------------------------------


C:\Users\Kamil\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  import sys


         * non-water            = 90.7% |  96.3%  | 85.8%
         * water                = 47.7% |  36.0%  | 70.9%


#### TAKEAWAY: 

Of course these results are biased bc done on top of training data. Yet it is nice to see that I gained 2 % becasue we dont have test data. More training data is can elevate the result. Now this model will be used to produce masks for MODIS 2000 - 2019 :)